In [ ]:
# Import libraries and set constants
from dotenv import load_dotenv
import os
import mdai
from mdai.visualize import load_mask, display_images
import json
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import random
import numpy as np

DEBUG = False

load_dotenv('dot.env')

ACCESS_TOKEN = os.getenv('MDAI_TOKEN')
DATA_DIR = os.getenv('DATA_DIR')
DOMAIN = os.getenv('DOMAIN')
PROJECT_ID = os.getenv('PROJECT_ID')
DATASET_ID = os.getenv('DATASET_ID')
ANNOTATIONS = os.path.join(DATA_DIR, os.getenv('ANNOTATIONS'))
LABEL_ID = os.getenv('LABEL_ID')

# Define an error threshold to filter out low-confidence points
ERROR_THRESHOLD = 1.0

print(f"ACCESS_TOKEN={ACCESS_TOKEN}")
print(f"DATA_DIR={DATA_DIR}")
print(f"DOMAIN={DOMAIN}")
print(f"PROJECT_ID={PROJECT_ID}")
print(f"DATASET_ID={DATASET_ID}")
print(f"ANNOTATIONS={ANNOTATIONS}")
print(f"LABEL_ID={LABEL_ID}")

# Start MD.ai client
mdai_client = mdai.Client(domain=DOMAIN, access_token=ACCESS_TOKEN)

# Download the dataset from MD.ai (or use cached version)
project = mdai_client.project(project_id=PROJECT_ID, path=DATA_DIR)

# Load the annotations
annotations_data = mdai.common_utils.json_to_dataframe(ANNOTATIONS)
annotations_df = pd.DataFrame(annotations_data['annotations'])
labels = annotations_df['labelId'].unique()

# Create the label map, LABEL_ID => 1, others in labels => 0
labels_dict = {LABEL_ID: 1}
project.set_labels_dict(labels_dict)

# Get the dataset
dataset = project.get_dataset_by_id(DATASET_ID)
dataset.classes_dict = project.classes_dict 

# Ensure BASE is set after preparing the dataset
BASE = dataset.images_dir

# Filter annotations for the free fluid label
free_fluid_annotations = annotations_df[annotations_df['labelId'] == LABEL_ID].copy()

# Function to construct the video path
def construct_video_path(base_dir, study_uid, series_uid):
    return os.path.join(base_dir, study_uid, f"{series_uid}.mp4")

# Add video paths to the dataframe using .loc to avoid the SettingWithCopyWarning
free_fluid_annotations['video_path'] = free_fluid_annotations.apply(
    lambda row: construct_video_path(BASE, row['StudyInstanceUID'], row['SeriesInstanceUID']), axis=1)

# Check if video files exist and add the result to the dataframe using .loc
free_fluid_annotations['file_exists'] = free_fluid_annotations['video_path'].apply(os.path.exists)

# Count the number of annotations with and without corresponding video files
num_with_files = free_fluid_annotations['file_exists'].sum()
num_without_files = len(free_fluid_annotations) - num_with_files

print(f"Annotations with corresponding video files: {num_with_files}")
print(f"Annotations without corresponding video files: {num_without_files}")

# Select five random annotations with corresponding video files
if DEBUG:
    random_annotations = free_fluid_annotations[free_fluid_annotations['file_exists']].sample(n=1, random_state=42)
else:
    random_annotations = free_fluid_annotations[free_fluid_annotations['file_exists']].sample(n=5, random_state=42)

# Display function
def polygons_to_mask(polygons, height, width):
    mask = np.zeros((height, width), dtype=np.uint8)
    for polygon in polygons:
        points = np.array(polygon, dtype=np.int32)
        cv2.fillPoly(mask, [points], 1)
    return mask

def display_annotation(row):
    video_path = row['video_path']
    frame_number = int(row['frameNumber'])
    foreground = row['data']['foreground']
    video_id = row['SeriesInstanceUID']

    cap = cv2.VideoCapture(video_path)
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
    ret, frame = cap.read()

    if not ret:
        print(f"Failed to read the frame number {frame_number} from the video.")
        return

    mask = polygons_to_mask(foreground, frame.shape[0], frame.shape[1])

    annotated_frame = frame.copy()
    annotated_frame[mask == 1] = (0, 0, 255)

    fig, ax = plt.subplots(1, 2, figsize=(15, 7))
    ax[0].imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    ax[0].set_title(f'Video ID: {video_id}')
    ax[1].imshow(cv2.cvtColor(annotated_frame, cv2.COLOR_BGR2RGB))
    # ax[1].set_title(f'Annotated Frame (Video ID: {video_id}')
    plt.show()

In [ ]:
# Parameters for Lucas-Kanade optical flow
lk_params = dict(winSize=(15, 15), maxLevel=2, criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Parameters for ShiTomasi corner detection
feature_params = dict(maxCorners=100, qualityLevel=0.3, minDistance=7, blockSize=7)

# Mask size and intensity thresholds
MASK_MIN_SIZE = 100
INTENSITY_THRESHOLD = 30

def apply_optical_flow(prev_frame, curr_frame, prev_mask):
    # Calculate dense optical flow
    flow = cv2.calcOpticalFlowFarneback(prev_frame, curr_frame, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    
    # Create a meshgrid of coordinates
    h, w = prev_frame.shape[:2]
    y, x = np.mgrid[0:h, 0:w].reshape(2, -1).astype(int)
    
    # Apply the flow to the coordinates
    fx, fy = flow[y, x].T
    coords = np.vstack([x + fx, y + fy]).round().astype(int)
    
    # Clip the coordinates to stay within the image
    coords[0] = np.clip(coords[0], 0, w - 1)
    coords[1] = np.clip(coords[1], 0, h - 1)
    
    # Create the new mask
    new_mask = np.zeros_like(prev_mask, dtype=float)
    new_mask[coords[1], coords[0]] = prev_mask[y, x]
    
    # Apply some morphological operations to clean up the mask
    kernel = np.ones((5,5), np.uint8)
    new_mask = cv2.morphologyEx(new_mask, cv2.MORPH_CLOSE, kernel)
    new_mask = cv2.morphologyEx(new_mask, cv2.MORPH_OPEN, kernel)
    
    return new_mask

def trim_and_threshold_mask(frame, mask):
    # Invert the frame so that dark areas (potential fluid) have high values
    inverted_frame = 255 - frame
    
    # Create a binary mask based on the intensity threshold
    intensity_mask = inverted_frame > INTENSITY_THRESHOLD
    
    # Combine the optical flow mask with the intensity mask
    trimmed_mask = (mask > 0) & intensity_mask
    
    # Check if the mask is too small
    if np.sum(trimmed_mask) < MASK_MIN_SIZE:
        return np.zeros_like(mask)
    
    return trimmed_mask.astype(float)

def track_frames(cap, start_frame, end_frame, initial_mask, forward=True):
    frames = []
    step = 1 if forward else -1
    frame_idx = start_frame
    
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
    ret, prev_frame = cap.read()
    if not ret:
        return frames
    
    prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    mask = initial_mask.astype(float)
    
    while (forward and frame_idx <= end_frame) or (not forward and frame_idx >= 0):
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
        ret, frame = cap.read()
        if not ret:
            break
        
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        # For the initial frame, use the initial mask
        if frame_idx == start_frame:
            new_mask = mask
        else:
            # Apply optical flow
            new_mask = apply_optical_flow(prev_gray, frame_gray, mask)
        
        # Trim low-intensity pixels and apply size threshold
        new_mask = trim_and_threshold_mask(frame_gray, new_mask)
        
        # Create annotated frame
        mask_overlay = np.zeros_like(frame, dtype=float)
        mask_overlay[:,:,1] = new_mask
        annotated_frame = cv2.addWeighted(frame, 1, (mask_overlay * 255).astype(np.uint8), 0.5, 0)
        
        # Add mask area text to the frame
        mask_area = np.sum(new_mask > 0)
        if DEBUG:
            cv2.putText(annotated_frame, f"Mask area: {mask_area}", (10, 30), 
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        
        frames.append((frame_idx, annotated_frame, new_mask))
        
        prev_gray = frame_gray.copy()
        mask = new_mask
        frame_idx += step
    
    return frames

def save_combined_video(video_path, output_video_path, initial_mask, frame_number):
    save_dir = os.path.dirname(output_video_path)
    mask_dir = os.path.join(save_dir, "masks")
    os.makedirs(mask_dir, exist_ok=True)
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # Track backward from the annotated frame to the start
    backward_frames = track_frames(cap, frame_number, 0, initial_mask, forward=False)
    # Track forward from the annotated frame to the end
    forward_frames = track_frames(cap, frame_number, total_frames - 1, initial_mask, forward=True)
    
    # Combine backward and forward frames
    combined_frames = backward_frames[::-1] + forward_frames[1:]
    
    # Get video properties
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))
    
    # Write combined frames to the video
    for frame_idx, frame, mask in combined_frames:
        out.write(frame)
        
        # Save mask as separate image
        mask_filename = os.path.join(mask_dir, f"mask_{frame_idx:04d}.png")
        cv2.imwrite(mask_filename, (mask * 255).astype(np.uint8))
    
    cap.release()
    out.release()
    print(f"Video saved at {output_video_path}")

def track_and_save_masks_as_video(annotation, output_dir):
    video_id = annotation['SeriesInstanceUID']
    video_path = annotation['video_path']
    frame_number = int(annotation['frameNumber'])
    foreground = annotation['data']['foreground']

    print(f"Processing Video: {video_id}; Frame: {frame_number}...")
    
    cap = cv2.VideoCapture(video_path)
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
    ret, frame = cap.read()
    if not ret:
        print(f"Failed to read the frame number {frame_number} from the video.")
        return
    cap.release()
    
    initial_mask = polygons_to_mask(foreground, frame.shape[0], frame.shape[1])
    print(f"Initial mask shape: {initial_mask.shape}")
    print(f"Initial mask min: {initial_mask.min()}, max: {initial_mask.max()}")
    cv2.imwrite(os.path.join(output_dir, f'initial_mask_frame_{frame_number}.png'), initial_mask * 255)
    
    output_video_path = os.path.join(output_dir, f'masked_{video_id}.mp4')
    save_combined_video(video_path, output_video_path, initial_mask, frame_number)


# Apply the tracking function to each annotation in random_annotations
output_base_dir = 'optical_flow'  # Base directory to save the tracked videos
os.makedirs(output_base_dir, exist_ok=True)

for index, annotation in random_annotations.iterrows():
    output_dir = os.path.join(output_base_dir, f'annotation_{index}')
    os.makedirs(output_dir, exist_ok=True)
    track_and_save_masks_as_video(annotation, output_dir)

print("Tracking and saving videos completed.")
